Camera Calibration with OpenCV
===



In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt


#declare parameters

nRow= 7
nCol= 9
dimension=0.02   #unit in meter

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((nRow*nCol,3), np.float32)
objp[:,:2] = (np.mgrid[0:9, 0:7].T.reshape(-1,2))*dimension

#objp[:,:2]=np.mgrid[0:180:20,0:140:20].T.reshape(-1,2)   # create real world coordinate of (0,0,0), (20,0,0),(40,0,0)..




criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, dimension, 0.001)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('/home/tanbir/Pictures/Webcam/side_camera_still_camera/*.jpg')
nPatternFound = 0
# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,7), None)

    # If found, add object points, image points
    if ret == True:
        
         #--- Sometimes, Harris cornes fails with crappy pictures, so
        #corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        nPatternFound += 1
        objpoints.append(objp)
        imgpoints.append(corners)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (9,7), corners, ret)
        #write_name = 'corners_found'+str(idx)+'.jpg'
        #cv2.imwrite(write_name, img)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# Below calibrates the camera

In [ ]:
scene=1

if (nPatternFound > 1):
    
    print("Found %d good images",nPatternFound)

    # Undistort an image
    if scene==1:
        
        img = cv2.imread('/home/tanbir/Pictures/Screenshot from video_1_181120132427.mkv.png')
        h,  w = img.shape[:2]
   # print "Image to undistort: ", imgNotGood
        newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
        print('scene 1 camera matrix :',newcameramtx)

    else:
        
        img = cv2.imread('/home/tanbir/Pictures/Screenshot from video.2.190207112801.video.mp4.png')
        h,  w = img.shape[:2]
   # print "Image to undistort: ", imgNotGood
        newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
        print('scene 0 camera matrix :',newcameramtx)
    # undistort
    mapx,mapy = cv2.initUndistortRectifyMap(mtx,dist,None,newcameramtx,(w,h),5)
    dst = cv2.remap(img,mapx,mapy,cv2.INTER_LINEAR)

    # crop the image
   # x,y,w,h = roi
   # dst = dst[y:y+h, x:x+w]
   # print "ROI: ", x, y, w, h

    cv2.imwrite('/home/tanbir/Pictures'+ "/calibresult.png",dst)
   # print "Calibrated picture saved as calibresult.png"
   # print "Calibration Matrix: "
   # print mtx
   # print "Disortion: ", dist

    #--------- Save result
    filename = '/home/tanbir/Pictures' + "/cameraMatrix.txt"
    np.savetxt(filename, mtx, delimiter=',')
    filename = '/home/tanbir/Pictures' + "/cameraDistortion.txt"
    np.savetxt(filename, dist, delimiter=',')

    print('camera matrix :',mtx)
    #print('othe camera matrix :',newcameramtx)
    mean_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
        error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
        mean_error += error

    print("total error: ", mean_error/len(objpoints))


# Camera Intrinsic and Extrinsic Matrix are calculated

In [ ]:
import numpy as np

dist= np.float64([[ 0.15934017, -0.30815065, -0.00506551, -0.00711125,  0.1484886 ]])


fx_new=(1280/1920)*1.19769086e+03   #camera was calibrated with 1920*1080 dimension so camera matrix modified for 1280*720 dimension
fy_new=(720/1080)*1.19035238e+03

new_camera_matrix= [[fx_new,0.00000000e+00,9.25663374e+02],[0.00000000e+00,fy_new,5.21621049e+02],[0.00000000e+00,0.00000000e+00,1.00000000e+00]]

print(new_camera_matrix)

In [ ]:
import numpy as np
import cv2
from numpy.linalg import inv

#camera_matrix = np.eye(3)
#camera_matrix= mtx
#dist_coeff = np.zeros(5)
dist_coeff= dist


# corresponding pixel coordinates for optitrack points

image = np.float64([[327,259],[869,242],[324,537],[880,518],[411,608]])   #M3 M4 M1 M2 origin order used

#optitrack points in csl lab

optitrack = np.float64([[1.1627,-0.0269,0],[1.1705,1.5567,0],[0.3855,-0.0248,0],[0.3898,1.5559,0],[0,0,0]])





retval, rvec, tvec = cv2.solvePnP(optitrack,image,np.asarray(new_camera_matrix), np.asarray(dist_coeff))


rotMat, _ = cv2.Rodrigues(rvec)

print(rotMat)


def groundProjectPoint(image_point, z = 0.0):
    
    camMat = np.asarray(new_camera_matrix)
    iRot = inv(rotMat)
    iCam = inv(camMat)

    uvPoint = np.ones((3, 1))

    # Image point
    uvPoint[0, 0] = image_point[0]
    uvPoint[1, 0] = image_point[1]

    tempMat = np.matmul(np.matmul(iRot, iCam), uvPoint)
    tempMat2 = np.matmul(iRot, tvec)

    s = (z + tempMat2[2, 0]) / tempMat[2, 0]
    wcPoint = np.matmul(iRot, (np.matmul(s * iCam, uvPoint) - tvec))

    # wcPoint[2] will not be exactly equal to z, but very close to it
    assert int(abs(wcPoint[2] - z) * (10 ** 8)) == 0
    wcPoint[2] = z

    return wcPoint

pixel = (395,389)
print("Pixel: %s" % (pixel, ))
print(groundProjectPoint(pixel))


# View Distorted and Undistorted Image

In [ ]:


import pickle
%matplotlib inline

# Test undistortion on an image
img = cv2.imread('/home/tanbir/Pictures/Screenshot.png')
img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
#ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)


dst = cv2.undistort(img, new_camera_matrix, dist, None, mtx)
cv2.imwrite('/home/tanbir/Pictures/test_undist3.jpg',dst)


f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)